In [86]:
import numpy as np
import math

from sympy import Matrix
from sympy.physics.quantum import TensorProduct

In [87]:
# Finds the index where a linear system is violated, returns the row which is in violation

def violation(A,b,x):
    check = (A@x >= b)
    
    for box in range(len(check)):
        if check[box][0] == False:
            return box

In [110]:
# https://webdocs.cs.ualberta.ca/~zacharyf/courses/combopt_2016/notes/lec21.pdf
def violation_cut(D,a):
    a_horz = a
    a_vert = a.reshape(-1,1)
    print(a)
    
    num_product = D@a_vert
    denom_product =  a_horz@num_product
    
    print(denom_product, '\n')
    
    b= (1/math.sqrt(denom_product[0]))*num_product
    
    return b

In [111]:
# Gives a new center for one iteration of the ellipsoid method

def recenter(n, center, cut):
    center = center + (1/(n+1))*cut
    print(center, '\n')
    return center

In [112]:
# Gives a matrix defining the new shape of the ellipsoid after one iteration of ellipsoid method

def reshape(n, D, a, cut):
    b = cut
    b_trans = np.swapaxes(b,0,1)
    
    shape = (math.pow(n,2)/(math.pow(n,2)-1)) * (D - (2/(n-1))*b@b_trans)
    print(shape, '\n \n')
    
    return shape

In [113]:
# Goes through one iteration of the main loop of the ellipsoid method
# A and b define the polygon P we are testing for, P := {x: Ax >= b}
# center and D define the ellipsoid searching for P
# n is the dimension of the system

def Ellipsoid_Iter(A,b,center,D,n):
    # Checks to see if current center is feasible
    test = (A@center >= b)
    
    # If center is feasible
    if np.all(test):
        return print(f'The point {center} is feasible!')
    
    # If center is infeasible
    else:
        index = violation(A,b,center)
        a = np.array(A[index])
        
        cut = violation_cut(D,a)
        center = recenter(n, center, cut)
        D = reshape(n, D, a, cut)
        
        new_ellipsoid = [center, D]
        
    return new_ellipsoid

In [114]:
# Goes through the ellipsoid Method

def Ellipsoid(A,b):
    U = max(A.max(),b.max()) # Largest value defining P
    n = float(len(b)) # Dimension of the system
    
    small_v = np.power(n,-n) * np.power((n*U), (-n**2 * (n+1))) # minimum possible volume of P
    big_V = np.power(2*n,n) * np.power(n*U, n**2) # Maximum possible volume of P
    
    time_limit = math.ceil(2*(n+1) * math.log(big_V/small_v, 2)) # Time Limit
    radius = big_V/math.pi # Radius squared of first unit ball
    D = radius * np.identity(int(n)) # Defining matrix of first ellipsoid
    center = np.zeros((int(n),1)) # First center set at 0
    time = 0
    data = [center,D]
    print(center, '\n', D, '\n \n')
    
    while time != time_limit:
        #print(data[0],'; \n', data[1], '\n \n')
        data = Ellipsoid_Iter(A,b,center,D,n)
        
        # If data type isn't a list, then we end the loop
        if type(data) != type([1,2,3]): 
            return('Process Ended')
        
        # If data is a list, recenter and reshape
        center = data[0]
        D = data[1]
        time = time+1
    
    
    return print('P is empty')

In [115]:
A = np.array([[-1,1],
              [1,0]])

b = np.array([[1],
              [1]])

Ellipsoid(A,b)

[[0.]
 [0.]] 
 [[81.48733086  0.        ]
 [ 0.         81.48733086]] 
 

[-1  1]
[162.97466173] 

[[-2.12769216]
 [ 2.12769216]] 

[[1.89478063e-14 1.08649774e+02]
 [1.08649774e+02 1.89478063e-14]] 
 

[1 0]
[1.89478063e-14] 

[[-2.12769212e+00]
 [ 2.63104399e+08]] 

[[-2.52637417e-14 -1.44866366e+02]
 [-1.44866366e+02 -1.66137417e+18]] 
 

[1 0]
[-2.52637417e-14] 



ValueError: math domain error